<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/b2b-analyis/blob/feat/B2B_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [1]:
#importar bibliotecas
import pandas as pd
import numpy as np

# bibliotecas para trabalhar offline com a biblioteca
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

#warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Para renderizar o plotly no colab

import plotly.io as pio
# pio.renderers
pio.renderers.default = 'colab'

# Carregar arquivos

In [3]:
# descrever o que é o arquivo
df_hist = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/historico_completo.csv",
                      parse_dates=['data'])


In [4]:
df_hist.data.max()

Timestamp('2023-12-31 00:00:00')

# Funções

In [5]:
def mudar_tipo_de_dados(dataframe, tipos_de_dados):
    """
    Muda o tipo de dados de colunas em um DataFrame.

    Parâmetros:
    - dataframe: DataFrame Pandas
    - tipos_de_dados: Lista de pares (nome da coluna, tipo de dados)
    """

    for coluna, tipo in tipos_de_dados:
        if coluna in dataframe.columns:
            dataframe[coluna] = dataframe[coluna].astype(tipo)

## Mudar tipo de dado

In [6]:
tipos_de_dados = [
    ('gerente_id', object),
    ('categoria_id', object),
    ('supervisor_id', object),
    ('funcionario_id', object),
    ('vendedor_id', object),
    ('produtto_id', object),
    ('itens_venda_id', object),
    ('cliente_id', object),
    ('vendas_id', object),
    ('geografia_id', object)
    ]

In [7]:
mudar_tipo_de_dados(df_hist, tipos_de_dados)

In [8]:
df_hist.data.min(), df_hist.data.max()

(Timestamp('2021-02-16 00:00:00'), Timestamp('2023-12-31 00:00:00'))

## Criar coluna ano-mes

In [ ]:
df_hist['ano_mes'] = df_hist['data'].dt.to_period('M')

## Tira teima

In [9]:
df_hist.query("gerente_id==1 & categoria_id ==4 & data >'2022-12-31' & data < '2023-04-01'").sort_values('data').set_index('data').resample('M')['meta_mensal'].first().sum()

70000.0

# EDA

Quantas unidades foram vendidas durante o todo o período?

2) preço médio unitário

In [10]:
preco_medio_unit = round(df_hist['valor_unitario'].mean(),2)
preco_medio_unit

151.71

3) Qual faturamento líquido

In [11]:
faturamento_liquido = round((df_hist['item_quantidade'] * df_hist['valor_unitario'] - df_hist['valor_desconto']).sum(),2)
faturamento_liquido

8611082.84

4) ticket medio

In [12]:
nfe_unicos = df_hist['nfe'].nunique()

ticket_medio = round(faturamento_liquido/nfe_unicos,2)
ticket_medio

2509.79

5) O que vendeu mais em cada ano

In [69]:
vendas_anual = (
    df_hist
    .groupby([df_hist['data'].dt.to_period('M'), 'categoria_descricao', 'produto_descricao'])
    .agg(
        quantidade=('item_quantidade', 'sum'),
        valor=('valor_venda', 'sum')
        )
    .reset_index()
    ).sort_values(['data', 'valor'], inplace=True)

In [73]:
vendas_anual['rank'] = vendas_anual.groupby('data')['valor'].rank(ascending=False)
vendas_anual.sort_values(['data', 'rank'], inplace=True)

vendas_anual.head()

In [94]:
top_cinco_por_ano = (
    vendas_anual
    .sort_values(['data', 'rank'])
    .groupby('data')
    .apply(lambda x: x.head(5))
)

top_cinco_por_ano


data categoria_descricao produto_descricao  quantidade  \
data                                                                     
2021-02 2    2021-02            Camiseta     Incomparaveis       198.0   
        11   2021-02         Manga Curta           Oriente        38.0   
        9    2021-02         Manga Curta           Intense        35.0   
        6    2021-02         Manga Curta            Cigano        30.0   
        12   2021-02         Manga Curta           Paeonia        31.0   
...              ...                 ...               ...         ...   
2023-10 810  2023-10         Manga Curta            Lilium        82.0   
        802  2023-10            Camiseta     Incomparaveis       531.0   
        804  2023-10         Manga Curta         Canadense        75.0   
        811  2023-10         Manga Curta           Oriente        62.0   
        808  2023-10         Manga Curta          Havaiana        72.0   

                 valor  rank  
data                          
2021-02 2     82172.60   1.0  
        11    69257.95   2.0  
        9     68190.35   3.0  
        6     64049.65   4.0  
        12    58538.65   5.0  
...                ...   ...  
2023-10 810  353768.50   1.0  
        802  347312.95   2.0  
        804  329307.50   3.0  
        811  328855.75   4.0  
        808  302558.55   5.0  

[165 rows x 6 columns]

In [87]:
#verificando o rank geral por categorias
rank_categorias = df_hist.groupby([df_hist['data'].dt.to_period('M'), 'categoria_descricao']).agg(
    quantidade=('item_quantidade', 'sum'),
    valor=('valor_venda', 'sum')).reset_index().sort_values(['data', 'valor'])
rank_categorias['rank'] = rank_categorias.groupby('data')['valor'].rank(ascending=False)
rank_categorias

,data,categoria_descricao,quantidade,valor,rank
0,2021-02,Blazer,44.0,80010.55,4.0
1,2021-02,Camiseta,198.0,82172.60,3.0
3,2021-02,Manga Longa,206.0,315742.55,2.0
2,2021-02,Manga Curta,302.0,562023.95,1.0
5,2021-03,Camiseta,531.0,277707.40,4.0
...,...,...,...,...,...
126,2023-09,Manga Curta,853.0,2897960.90,1.0
129,2023-10,Camiseta,531.0,347312.95,4.0
128,2023-10,Blazer,115.0,556097.20,3.0
131,2023-10,Manga Longa,471.0,1988085.95,2.0


In [95]:
#Top cinco por categoria e por ano
top_cinco_por_categoria = (
    rank_categorias
    .sort_values(['data', 'rank'])
    .groupby('data')
    .apply(lambda x: x.head(5))
)

top_cinco_por_categoria

,data,categoria_descricao,quantidade,valor,rank
2,2021-02,Manga Curta,302.0,562023.95,1.0
3,2021-02,Manga Longa,206.0,315742.55,2.0
1,2021-02,Camiseta,198.0,82172.60,3.0
0,2021-02,Blazer,44.0,80010.55,4.0
6,2021-03,Manga Curta,789.0,1808273.00,1.0
...,...,...,...,...,...
125,2023-09,Camiseta,561.0,373410.70,4.0
130,2023-10,Manga Curta,726.0,3124550.40,1.0
131,2023-10,Manga Longa,471.0,1988085.95,2.0
128,2023-10,Blazer,115.0,556097.20,3.0


6) trabalhar com metas

In [27]:
(
    df_hist
    .groupby(['gerente_id', 'categoria_id', df_hist['data'].dt.to_period('M'),])
    .agg(
        meta_mensal=('meta_mensal','first'),
        meta_diaria_equipe=('$_Meta_por_dia_util', 'first'),
        meta_diaria_ind=('meta_diaria_ind', 'first')
        )
    .reset_index()

)

,gerente_id,categoria_id,data,meta_mensal,meta_diaria_equipe,meta_diaria_ind
0,1,1,2021-02,NaN,NaN,NaN
1,1,1,2021-03,NaN,NaN,NaN
2,1,1,2021-04,NaN,NaN,NaN
3,1,1,2021-05,NaN,NaN,NaN
4,1,1,2021-06,NaN,NaN,NaN
...,...,...,...,...,...,...
275,2,4,2023-08,17692.307692,769.230769,128.21
276,2,4,2023-09,16153.846154,769.230769,128.21
277,2,4,2023-10,23692.307692,1076.923077,179.49
278,2,4,2023-11,23692.307692,1076.923077,179.49


In [38]:
#vendas realizadas por gerente e por vendedor

df_hist.groupby(['gerente_id', 'vendedor_id','data' , 'nfe']).agg(
    meta_mensal_equipe = ('meta_mensal', 'first'),
    meta_diaria = ('meta_diaria_ind', 'first'),
    valor_realizado_dia = ('valor_venda', 'sum')
).reset_index()

#.query("gerente_id==1 & vendedor_id ==1 & data >'2021-12-31' & data < '2022-02-01'")

,gerente_id,vendedor_id,data,nfe,meta_mensal_equipe,meta_diaria,valor_realizado_dia
0,1,1.0,2021-02-22,SO43690,NaN,NaN,2556.4
1,1,1.0,2021-02-27,SO43869,NaN,NaN,12541.5
2,1,1.0,2021-03-02,SO43881,NaN,NaN,9046.1
3,1,1.0,2021-03-04,SO43888,NaN,NaN,31897.6
4,1,1.0,2021-03-04,SO43890,NaN,NaN,16798.1
...,...,...,...,...,...,...,...
3426,2,10.0,2023-10-08,SO97066,28769.230769,217.95,193966.5
3427,2,10.0,2023-10-13,SO97362,88846.153846,673.08,36918.0
3428,2,10.0,2023-10-15,SO97368,88846.153846,673.08,48520.5
3429,2,10.0,2023-10-19,SO117674,88846.153846,673.08,5070.9


In [37]:
#Verificar os valores das metas e o fator vendedor

df_hist.groupby(['gerente_id', 'categoria_id','vendedor_id','data' , 'nfe']).agg(
    meta_mensal_equipe = ('meta_mensal', 'first'),
    meta_diaria_ind = ('meta_diaria_ind', 'first'),
    valor_realizado_dia = ('valor_venda', 'sum')
).reset_index()

,gerente_id,categoria_id,vendedor_id,data,nfe,meta_mensal_equipe,meta_diaria_ind,valor_realizado_dia
0,1,1,1.0,2021-02-27,SO43869,NaN,NaN,4180.50
1,1,1,1.0,2021-03-04,SO43888,NaN,NaN,6835.20
2,1,1,1.0,2021-03-04,SO43890,NaN,NaN,15271.00
3,1,1,1.0,2021-03-04,SO43891,NaN,NaN,3455.85
4,1,1,1.0,2021-03-13,SO44095,NaN,NaN,1379.60
...,...,...,...,...,...,...,...,...
9078,2,4,10.0,2023-09-27,SO97028,16153.846154,128.21,6549.00
9079,2,4,10.0,2023-09-28,SO97032,16153.846154,128.21,2050.25
9080,2,4,10.0,2023-10-08,SO97066,23692.307692,179.49,6465.55
9081,2,4,10.0,2023-10-13,SO97362,23692.307692,179.49,3076.50


In [36]:
df_hist.query("nfe=='SO97368'")

,vendas_id,data,nfe,cliente_id,nf_desconto,valor_liquido,valor_venda,itens_venda_id,produto_id,item_quantidade,...,supervisor_id,gerente_id,categoria_id,produto_descricao,tamanho,custoUnitario,categoria_descricao,meta_mensal,$_Meta_por_dia_util,meta_diaria_ind
718,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54229.0,49.0,1.0,...,5.0,2,2,Roseta,GG,115.0,Manga Curta,88846.153846,4038.461538,673.08
1535,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54227.0,36.0,1.0,...,5.0,2,2,Lilium,G,110.0,Manga Curta,88846.153846,4038.461538,673.08
5793,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54219.0,71.0,2.0,...,5.0,2,2,Canadense,M,105.0,Manga Curta,88846.153846,4038.461538,673.08
6408,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54222.0,54.0,2.0,...,5.0,2,2,Intense,P,125.0,Manga Curta,88846.153846,4038.461538,673.08
7000,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54233.0,88.0,1.0,...,5.0,2,2,Beach Flash,GG,80.0,Manga Curta,88846.153846,4038.461538,673.08
10711,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54226.0,91.0,9.0,...,5.0,2,4,Incomparaveis,M,55.0,Camiseta,23692.307692,1076.923077,179.49
14599,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54223.0,37.0,1.0,...,5.0,2,2,Lilium,GG,110.0,Manga Curta,88846.153846,4038.461538,673.08
18526,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54221.0,79.0,1.0,...,5.0,2,2,Porto Rico,P,110.0,Manga Curta,88846.153846,4038.461538,673.08
19313,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54232.0,40.0,1.0,...,5.0,2,3,Blazer Slim Fit,GG,200.0,Blazer,22000.000000,1000.000000,166.67
24346,3328.0,2023-10-15,SO97368,442.0,179.3,3414.0,3234.7,54220.0,75.0,1.0,...,5.0,2,2,Cigano,GG,115.0,Manga Curta,88846.153846,4038.461538,673.08


In [39]:
# Agrupe os dados por 'gerente_id', 'vendedor_id' e ano da data
df_agrupado = df_hist.groupby(['gerente_id', 'vendedor_id', df_hist['data'].dt.year])

# Em seguida, calcule a soma das colunas 'meta_mensal' e 'valor_liquido' para cada grupo
resultado = df_agrupado.agg(
    Soma_Meta_Mensal_Geral=('meta_mensal', 'sum'),
    Soma_Valor_Realizado=('valor_venda', 'sum')
)

# Exiba o resultado
print(resultado)

                             Soma_Meta_Mensal_Geral  Soma_Valor_Realizado
gerente_id vendedor_id data                                              
1          1.0         2021            0.000000e+00            4868589.25
                       2022            3.483045e+07            6630665.85
                       2023            3.777029e+07            3447412.05
           4.0         2021            0.000000e+00            4414707.00
                       2022            2.566874e+07            5391070.85
                       2023            3.836213e+07            3389534.45
           6.0         2021            0.000000e+00            6054471.25
                       2022            1.987034e+07            3403760.60
                       2023            5.743794e+07            5592883.45
           7.0         2021            0.000000e+00            5303757.35
                       2022            2.758712e+07            5800872.25
                       2023           

In [41]:

df_filtrado = df_hist.groupby(['gerente_id', 'vendedor_id', 'ano_mes']).first().reset_index()


resultado = df_filtrado.groupby(['gerente_id', 'vendedor_id', df_filtrado['data'].dt.year]).agg(
    Soma_Meta_Mensal_Geral=('meta_mensal', 'sum'),
    soma_meta_individual=('meta_diaria_ind','sum'),
    Soma_Valor_Realizado=('valor_venda', 'sum')
)

# Exiba o resultado
resultado

Soma_Meta_Mensal_Geral  soma_meta_individual  \
gerente_id vendedor_id data                                                 
1          1.0         2021                0.000000                  0.00   
                       2022           579420.000000               4445.32   
                       2023           698615.384615               5358.98   
           4.0         2021                0.000000                  0.00   
                       2022           598203.000000               4587.62   
                       2023           698615.384615               5358.98   
           6.0         2021                0.000000                  0.00   
                       2022           531733.000000               4084.05   
                       2023           698615.384615               5358.98   
           7.0         2021                0.000000                  0.00   
                       2022           550680.000000               4227.59   
                       2023           698615.384615               5358.98   
           11.0        2021                0.000000                  0.00   
                       2022           561541.000000               4315.76   
                       2023           642000.000000               4948.72   
           12.0        2021                0.000000                  0.00   
                       2022           542022.000000               4155.44   
                       2023           698615.384615               5358.98   
2          2.0         2021                0.000000                  0.00   
                       2022           600142.000000               4622.27   
                       2023           576346.153846               4423.11   
           3.0         2021                0.000000                  0.00   
                       2022           466631.000000               3562.34   
                       2023           576346.153846               4423.11   
           5.0         2021                0.000000                  0.00   
                       2022           602618.000000               4633.06   
                       2023           539961.538462               4147.46   
           8.0         2021                0.000000                  0.00   
                       2022           558196.000000               4289.32   
                       2023           576346.153846               4423.11   
           9.0         2021                0.000000                  0.00   
                       2022           558196.000000               4289.32   
                       2023           576346.153846               4423.11   
           10.0        2021                0.000000                  0.00   
                       2022           597607.000000               4586.85   
                       2023           539961.538462               4147.46   

                             Soma_Valor_Realizado  
gerente_id vendedor_id data                        
1          1.0         2021              48667.80  
                       2022              62591.70  
                       2023              39569.05  
           4.0         2021              39290.25  
                       2022              50809.15  
                       2023              40648.50  
           6.0         2021              50535.00  
                       2022              51725.60  
                       2023              54192.00  
           7.0         2021              45839.90  
                       2022              48567.95  
                       2023              33464.75  
           11.0        2021              39970.30  
                       2022              59387.60  
                       2023              50550.95  
           12.0        2021              54609.65  
                       2022              57976.70  
                       2023              53005.70  
2          2.0         2021              46175.20  
          

In [42]:
# Localizar os valores NaN
valores_nan = df_hist.isna()

# filtrar as linhas com NaN
linhas_com_nan = df_hist[valores_nan.any(axis=1)]
linhas_com_nan

,vendas_id,data,nfe,cliente_id,nf_desconto,valor_liquido,valor_venda,itens_venda_id,produto_id,item_quantidade,...,gerente_id,categoria_id,produto_descricao,tamanho,custoUnitario,categoria_descricao,meta_mensal,$_Meta_por_dia_util,meta_diaria_ind,ano_mes
0,1.0,2021-02-16,SO43660,614.0,24.90,265.0,240.10,13.0,43.0,1.0,...,1,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN,2021-02
2,993.0,2021-06-24,SO46324,259.0,62.80,1014.0,951.20,7650.0,43.0,1.0,...,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN,2021-06
3,7.0,2021-02-17,SO43665,347.0,112.95,1711.0,1598.05,61.0,43.0,1.0,...,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN,2021-02
5,12.0,2021-02-18,SO43670,266.0,26.35,602.0,575.65,112.0,43.0,1.0,...,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN,2021-02
7,454.0,2021-04-26,SO44762,202.0,306.85,3776.0,3469.15,3627.0,43.0,1.0,...,1,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN,2021-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40515,NaN,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,4,NaN,NaN,NaN,NaN,22615.384615,1076.923077,179.49,2023-12
40516,NaN,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,4,NaN,NaN,NaN,NaN,22615.384615,1076.923077,179.49,2023-12
40517,NaN,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,4,NaN,NaN,NaN,NaN,22615.384615,1076.923077,179.49,2023-12
40518,NaN,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,4,NaN,NaN,NaN,NaN,22615.384615,1076.923077,179.49,2023-12
